In [1]:
import os
filename = os.path.join('example-data', 'iris', 'iris_1.csv')
filename

'example-data/iris/iris_1.csv'

In [2]:
import dask.dataframe as dd
df_csv = dd.read_csv(filename)
df_csv.head()

/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


,sepal_length,sepal_width,petal_length,petal_width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


# Write to HDF5
HDF5 is an binary array format. 

In [3]:
target = os.path.join('example-data', 'iris', 'iris_1.h5')
target

'example-data/iris/iris_1.h5'

In [4]:
# convert to binary format, takes some time up-front
%time df_csv.to_hdf(target, '/data')

CPU times: user 282 ms, sys: 158 ms, total: 440 ms
Wall time: 781 ms


['example-data/iris/iris_1.h5']

In [5]:
# same data as before
df_hdf = dd.read_hdf(target, '/data')
df_hdf.head()

,sepal_length,sepal_width,petal_length,petal_width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


# Compare CSV to HDF5 speeds

In [8]:
%time 
df_csv.sepal_length.sum().compute()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


876.5

In [9]:
%time 
df_hdf.sepal_length.sum().compute()


CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 4.77 µs


876.5

# Make HDF5 speeds even faster 
by saving HDF5 file, via calling a method to replace columns with string dtypes with numerical representation.

In [10]:
# Categorical data column is 'variety', then store in HDFStore
%time 
df_hdf.categorize(columns=['variety']).to_hdf(target, '/data2')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 2.86 µs


['example-data/iris/iris_1.h5']

In [11]:
# It looks the same
df_hdf = dd.read_hdf(target, '/data2')
df_hdf.head()

,sepal_length,sepal_width,petal_length,petal_width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [13]:
# But loads more quickly
%time df_hdf.sepal_length.sum().compute()

CPU times: user 30 ms, sys: 3.48 ms, total: 33.5 ms
Wall time: 33.5 ms


876.5

In [15]:
# conclusion, there was no speed increase from the suggested edits from the tutorial:
# https://tutorial.dask.org/07_dataframe_storage.html

# Maybe such techniques are more appropriate for larger datasets?